In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

In [2]:
!git pull --all --force
#!pip install peft==0.4.0
#!pip install  -U git+https://github.com/huggingface/peft.git
%cd /kaggle/working/chatGLM-6B-QLoRA 
!ls
!pip install -r requirements.txt
#!pip install deepspeed==0.9.5  这个也是需要的 但是目前kaggle 的runtime自带了

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/kaggle/working/chatGLM-6B-QLoRA
README.md
chatGLM_6B_QLoRA.json
data
ds_zero2_config.json
ds_zero3_config.json
inference_qlora.py
inference_test.py
luzi.json
merge_lora_and_quantize.py
pics
pretrain_chatglm2_v5.ipynb
pretrain_chatglm2_v7.ipynb
pretrain_chatglm2_v8.ipynb
pretrain_qlora_chatglm2.py
qa_aug.json
qlora_param.json
remote_scripts
requirements.txt
sft_multi_turn_qlora_chatglm2.py
train_ds_zero2_test.py——错误的
train_normal_lora.py
train_qlora.py
train_qlora_deepspeed_zero.py
二次预训练pt_sft_继续lora训练_qlora_glm_成功0730.ipynb
参考与经验


In [ ]:
ls

In [3]:
# ./data/sharegpt_multi_turn_data 目录不能有空白json文件
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  sft_multi_turn_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-sharegpt-sft-0802-v1 \
  --num_train_samples -1 \
  --num_eval_samples 50 \
  --train_data_path ./data/sharegpt_data/   \
  --eval_data_path  ./data/sharegpt_data    \
  --data_type sharegpt  \
  --max_length 1600 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  5e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

Fetching origin
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), 1.37 MiB | 4.96 MiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   b779455..a49e779  main       -> origin/main
Updating b779455..a49e779
Fast-forward
 data/sharegpt_data/sharegpt_zh_1K_format.jsonl     | 1000 ++++++++++++++++++++
 .../sharegpt_format_conv.json                      |    0
 2 files changed, 1000 insertions(+)
 create mode 100644 data/sharegpt_data/sharegpt_zh_1K_format.jsonl
 delete mode 100644 data/sharegpt_multi_turn_data/sharegpt_format_conv.json
[2023-08-02 13:28:39,315] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy 